### Logistic Classifier for CIFAR-10

In [1]:
import matplotlib.pyplot as plt
from data_loader import load_training, load_testing, load_validation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

from Cifar10 import Cifar10

In [2]:
# device configuration
# device = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))
device = torch.device('cpu')

print(f'Using device {device}')

# consitency in training
torch.manual_seed(1234)
random.seed(1234)

Using device cpu


In [3]:
# Load data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = Cifar10(train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = Cifar10(train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Hyperparameters for LogReg model:
- solver
- penalty

Constant for all clfs:
- C = 1/c (eliminates inversion)

Three models:
- liblinear, l1 (v1)
- liblinear, l2 (v2)
- sag, l2       (v3)

In [4]:
# define Logistic Classifier

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.log_layer = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = self.log_layer(x)
        return x


net = Net().to(device)

In [5]:
# Hyper Parameters
num_epochs = 1
learning_rate = 0.001

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [6]:
# train the model 
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # backward + optimze
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 3.642
[1,  4000] loss: 3.216
[1,  6000] loss: 3.089
[1,  8000] loss: 3.036
[1, 10000] loss: 2.952
Finished Training


In [7]:
# Evaluate
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(testset)} test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 35 %


In [3]:
# v1

v1_tr_errors = []
v1_te_errors = []
v1_val_errors = []

c_list = [0, 0.1, 1, 10, 50]

for c in c_list:
    if c == 0:
        # LogReg with no penalty
        clf = LogisticRegression(penalty=None)
    else:
        clf = LogisticRegression(penalty="l1", solver="liblinear", C=1/c)

    clf.fit(tr_X, tr_y)

    acc_tr = accuracy_score(tr_y, clf.predict(tr_X))
    acc_te = accuracy_score(te_y, clf.predict(te_X))
    v1_tr_errors.append(acc_tr)
    v1_te_errors.append(acc_te)

plt.title("Varying error rates on different regularization strength on v1")
plt.plot(c_list, v1_tr_errors, c="red", label="training")
plt.plot(c_list, v1_te_errors, c="green", label="testing")
plt.legend()
plt.xlabel("reg strength")
plt.ylabel("error rate")
plt.show()

print(v1_tr_errors)
print(v1_te_errors)


/Users/joshcordero/miniconda3/envs/cs178/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
